In [28]:
import pickle
import os 
import cv2
import glob
import matplotlib.pyplot as plt

# select one of the multiple GPU cards on the server.
os.environ["CUDA_VISIBLE_DEVICES"]="0" # set GPU if multiple present


In [29]:
'''
This code is developed and run on at least two machines
Here we setup the default paths for each machine.
Eventually we must find a more general way to do this.
'''

host = os.getenv('HOSTNAME')
print(host)

if "dsigpu" in host:
    pklDir = '/home/dsi/aryeh/data/mia/pkl/'
    modelDir = '/home/dsi/aryeh/data/mia/trained_models/'
    modelname = "unetDensnet201_2023-01-24_08-17-37"
    trainingdata = '/home/dsi/aryeh/data/plants/unCropped/resized/'
    predictionFolder = '/home/dsi/aryeh/data/plants/Harvest8Orange5_7Oct17/'
elif "pop" in host:
    pklDir = '/media/amw/TOSHIBA EXT/alerding/models/pkl/'
    modelDir = '/media/amw/TOSHIBA EXT/alerding/models/gpuModels/'
    modelname = "unetDensnet201_2023-01-24_08-17-37"
    trainingdata =  '/media/amw/TOSHIBA EXT/alerding/annotated/notCroppedLabels2/resized/'
    predictionFolder = '/media/amw/TOSHIBA EXT/alerding/Harvest 8 Orange 5,7 Oct 17.r/vertical/'
else:
    pklDir = None
    modelDir = None 
    modelname = "unetDensnet201_2023-01-24_08-17-37"
    trainingdata = None
    predictionFolder = None

dsigpu06


In [30]:
# the pkl directory contains the pkl files that hold a DeepLearning structure that defines the DL model
# If the dl object is directly created then the pkl fileis not needed.
pklDir = input('Enter pkl path or use default\n') or pklDir

# the model directory will receive the trained model
modelDir = input('Enter model output directory or default\n') or modelDir
    
loadweights = None

# modelname is the name of the pkl file without its pkl  extension.
# a pkl file holds a data structure or object. IN this case, it is expected to hold a DeepLearning object that was saved
# using File>Save DL Object... in mianalyzer. 
# If the dl object is directly created then this is not needed.
modelname = input("Enter modelname without extension or use default") or modelname

# stem, pop, background and unlabeled
# We should have a single cell where aall of the model and training parameters are set
numclasses = 4

# trainingdata is a directory that holds the annotated images. The labels must be in trainingdata/Segmentation_labels/
# and they must be npz files.
trainingdata = input("Path to folder containing training images, or use default\n") or trainingdata
validationdata = None

print("model: ", modelname)



model:  unetDensnet201_2023-01-24_08-17-37


In [ ]:
'''
This is from the version that reads the dl object from a pkl file.
Now that the  dl structure can be defined in the code, it is not required.
'''

print('load settings')
#filehandler = open(modelname + '.pkl', 'rb')
filehandler = open(pklDir + modelname + '.pkl', 'rb')

dl = pickle.load(filehandler)

from pprint import pprint

pprint(vars(dl))


In [31]:
'''
Creates the dl object. Default is UNet, resnet152, 100 epochs,
scaleFactor is 0.5, and learning rate is 0.001.
All the other parmeters are the defaults of the dl object.
'''

print(os.getcwd())


from dl.DeepLearning import *
def createDL( epochs=100, scaleFactor=0.5, learning_rate = 0.001):
    dl = DeepLearning()
    dl.epochs = epochs
    dl.ImageScaleFactor = scaleFactor
#    dl.Mode = Segmentation()
    dl.learning_rate = learning_rate
    dl.Mode.architecture = 'UNet'
    dl.Mode.backbone = 'resnet152'
    return dl

/home/dsi/aryeh/git/miaHeadless/mia


In [32]:
'''
create the dl object and verify that it is initialized. 
'''

dl = createDL( epochs=50, scaleFactor=0.5, learning_rate = 0.002)

# pod, stem, background, and unlabeled (for deleaved soybean plants)
numClasses =4

print('init model')
dl.initModel(numclasses)

dl.Model = dl.Mode.getModel(numClasses, 3)

print('load model')
if loadweights is not None:
    dl.Model.load_weights(loadweights)

print(dl.initialized)

init model
load model
True


In [33]:
'''
Here we  do the training
'''
        
if dl.initialized:
    print('start training')
    dl.initData_StartTraining(trainingdata, validationdata)
    print('training finished')
else:
    print('could not initialize model')
            
print('saving weights')
dl.Model.save_weights(modelDir+modelname + '.h5')
print('saving training data')
dl.saveTrainingRecord(modelDir+modelname +'.csv')
#   print('saving model')
#   dl.Model.save(modelDir+'full')
    



start training
Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
139/139 [==============================] - 139s 878ms/step - loss: 0.0205 - mean_iou: 0.2287 - val_loss: 0.0124 - val_mean_iou: 0.2581
Epoch 2/50
139/139 [==============================] - 116s 832ms/step - loss: 0.0140 - mean_iou: 0.2772 - val_loss: 0.1086 - val_mean_iou: 0.2498
Epoch 3/50
139/139 [==============================] - 117s 833ms/step - loss: 0.0135 - mean_i

In [34]:
%matplotlib inline

In [ ]:
# This cell does a prediction on a directory of jpg images (could be any type, but that is what I have).
# predictionFolder contains the images which ill be predicted, and a subdirectory with the modelname will be created
# to hold the predicted segmentation.

predictionFolder = input('Enter folder for prediction or use default\n')  or predictionFolder
outputPath = predictionFolder + modelname+'_ep'+str(dl.epochs)+'/'
try:
    os.mkdir(outputPath)
except FileExistsError:
    pass
files = glob.glob(predictionFolder + '*.jpg')
print(len(files))

# If we do not want to process the entire folder, we can limit the number of processed image to count. 
count = 0
for i in files:
    print(i)
    img = cv2.imread(i)
    prediction = dl.Mode.PredictImage(img)
    plt.figure(count)
    plt.imshow(prediction)
    plt.show()
    cv2.imwrite(outputPath+'segmented_'+os.path.basename(i).replace('jpg', 'png'), prediction)
    count += 1
    if count > 1:
       break
    



In [ ]:
# I was checking why repeated runs of the prediction produced different reauslts. One possibility is that
# I need to reset the DL object between runs. But I am not sure about this.
print(dl.Model)
dl.cleanMemory()